In [1]:
import argparse
import json
import os
import torch
import os.path
import sys
import time
from math import ceil, e, exp
import math

import numpy as np
import soundfile as sf

# Load (source) config file

In [2]:
config_fpath = r"H:\TTCheckpoints\waveglow\outdir_EfficientLarge3\config.json"

# Parse configs.  Globals nicer in this case
with open(config_fpath) as f:
    data = f.read()
config = json.loads(data)
train_config = config["train_config"]
global data_config
data_config = config["data_config"]
global dist_config
dist_config = config["dist_config"]
global waveglow_config
waveglow_config = {
    **config["waveglow_config"], 
    'win_length': data_config['win_length'],
    'hop_length': data_config['hop_length']
}
print(str(waveglow_config).replace(",",",\n"))

{'yoyo': True,
 'yoyo_WN': False,
 'n_mel_channels': 160,
 'n_flows': 48,
 'n_group': 24,
 'n_early_every': 8,
 'n_early_size': 2,
 'memory_efficient': True,
 'spect_scaling': False,
 'upsample_mode': 'normal',
 'WN_config': {'n_layers': 8,
 'n_channels': 256,
 'kernel_size': 3,
 'speaker_embed_dim': 0,
 'rezero': False},
 'win_length': 2400,
 'hop_length': 600}


# Initialise Blank Target Model

In [3]:
if waveglow_config["yoyo"]: # efficient_mode # TODO: Add to Config File
    from efficient_model import WaveGlow
    from efficient_loss import WaveGlowLoss
else:
    from glow import WaveGlow, WaveGlowLoss
model = WaveGlow(**waveglow_config).cuda()

# Load Checkpoint

In [5]:
checkpoint_path = r"H:\TTCheckpoints\waveglow\outdir_EfficientLarge3\waveglow_90065"

assert os.path.isfile(checkpoint_path)
checkpoint_dict = torch.load(checkpoint_path, map_location='cpu')
model_dict = checkpoint_dict['model']
if (str(type(model_dict)) != "<class 'collections.OrderedDict'>"):
    model_dict = model_dict.state_dict()

#model = model.cuda()

# Reshape the Weights

In [11]:
for key, weights in model_dict.items():
    print(weights.shape)
    print(key)
    break

torch.Size([24, 24, 1])
convinv.0.weight
